<a href="https://colab.research.google.com/github/chaitsugale/Stock-Market-Prediction-The-Spark-Foundation-/blob/main/Stock_Market_Prediction_Using_Numerical_and_Textual_Analysis(The_Spark_Foundation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Market Prediction Using Numerical and Textual Analysis

Objective: Create a hybrid model for stock price/performance
prediction using numerical analysis of historical stock prices, and
sentimental analysis of news headlines

Here We are analyzing and predicting the stocks of BOMBAY STOCK EXCHANGE i.e. SENSEX (S&F BSE SENSEX) 

SENSEX or BOMBAY STOCK EXCHANGE is the oldest stock exchange in ASIA establishing in 1875.

## Step 1: Loading the Package

> Loading all the important packages for analyzing and predicting alonng with plotting graph



In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import altair as alt  #Altair is a declarative statistical visualization library for Python

import statsmodels.api as sm 

from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import RandomForestRegressor

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# installing pypi package for yahoo finance


!pip install yfinance --upgrade --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
